# Pymaceuticals Inc.
---

### Analysis

- Write at least three observations or inferences that can be made from the data. Include these observations at the top of your notebook.

In [1]:
!pip install matplotlib


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

In [3]:
# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

mouse_metadata_df = pd.DataFrame(mouse_metadata)
study_results_df = pd.DataFrame(study_results)
study_results_df.head(-20)

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.000000,0
1,f932,0,45.000000,0
2,g107,0,45.000000,0
3,a457,0,45.000000,0
4,c819,0,45.000000,0
...,...,...,...,...
1868,r604,45,62.670759,3
1869,z969,45,73.867845,4
1870,g791,45,29.128472,1
1871,i635,45,58.633404,3


In [4]:
#Look at columns to find where to merge
mouse_metadata.columns


Index(['Mouse ID', 'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)'], dtype='object')

In [5]:
#Look at columsn to find where to merge
study_results.columns


Index(['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites'], dtype='object')

In [6]:
# Combine the data into a single dataset
mouse_study_df = pd.merge(study_results_df, mouse_metadata_df, on=['Mouse ID'])
copy_mouse_study = mouse_study_df

mouse_study_df.head()


,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [7]:
mouse_study_df.count()

Mouse ID              1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
Drug Regimen          1893
Sex                   1893
Age_months            1893
Weight (g)            1893
dtype: int64

In [8]:
#Display the number of unique mice IDs in the data, and then check for any mouse ID with duplicate time points.
number_of_mice = len(mouse_study_df['Mouse ID'].unique())
print(number_of_mice)


249


In [20]:
# Get list of duplicate rows based on 'Mouse ID' column
df2 = mouse_study_df[mouse_study_df.duplicated('Mouse ID')]
#print(df2)

# Select duplicate rows of all columns
df2 = mouse_study_df[mouse_study_df.duplicated("Timepoint")]
print(df2)



     Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites Drug Regimen  \
10       f932          0           45.000000                 0     Ketapril   
11       g107          0           45.000000                 0     Ketapril   
12       g107          5           48.791665                 0     Ketapril   
13       g107         10           53.435987                 0     Ketapril   
14       g107         15           58.135545                 0     Ketapril   
...       ...        ...                 ...               ...          ...   
1888     m601         25           33.118756                 1    Capomulin   
1889     m601         30           31.758275                 1    Capomulin   
1890     m601         35           30.834357                 1    Capomulin   
1891     m601         40           31.378045                 1    Capomulin   
1892     m601         45           28.430964                 1    Capomulin   

         Sex  Age_months  Weight (g)  
10      Male

In [21]:

#Duplicated Mouse ID dataframe
dup_data_df = pd.DataFrame(df2)
# Select duplicate rows of all columns

#dup_data_df.head()

In [22]:
#check to evaluate the numbers are the same.
mouse_study_df['Mouse ID'].duplicated().sum()


1644

In [31]:
#drop the duplicates
mouse_study_df2 = mouse_study_df.drop_duplicates()

mouse_study_df2 = mouse_study_df.drop_duplicates()
mouse_study_df2.head(500)

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22
...,...,...,...,...,...,...,...,...
495,j989,20,41.635344,1,Ramicane,Male,8,19
496,j989,25,39.934112,1,Ramicane,Male,8,19
497,j989,30,34.671781,1,Ramicane,Male,8,19
498,j989,35,35.115590,1,Ramicane,Male,8,19


In [32]:
number_of_mice = len(mouse_study_df['Mouse ID'].unique())
print(number_of_mice)

#mouse_study_df2[["Sex"]].describe()
#mouse_study_df2[["Mouse ID"]].describe()
#mouse_study_df2[["Drug Regimen"]].describe()

249


# FLAG--- should be 248

In [15]:
mouse_study_df2.count()


Mouse ID              10
Timepoint             10
Tumor Volume (mm3)    10
Metastatic Sites      10
Drug Regimen          10
Sex                   10
Age_months            10
Weight (g)            10
dtype: int64

### Summary Statistics


In [16]:
mouse_study_df2[["Tumor Volume (mm3)"]].describe()

,Tumor Volume (mm3)
count,10.000000
mean,41.963636
std,2.782135
min,37.967644
25%,39.388492
50%,42.996848
75%,43.656383
max,45.651331


In [17]:
#Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
mouse_study_stats_df = pd.DataFrame()
mouse_study_stats_df.sort_index(axis=1,ascending=True)

#Use groupby and summary statistical methods to calculate the following properties of each drug regimen:
#drug_regimen = mouse_study_df2.groupby(['Drug Regimen'])
mean_tumor_regimen = mouse_study_df2.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
median_tumor_volume = mouse_study_df2.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
tumor_volume__stddev = mouse_study_df2.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
tumor_volume_stderr = mouse_study_df2.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].sem()

#print(drug_regimen)
#mouse_study_stats_df['Drug Regimen'] = [drug_regimen]
mouse_study_stats_df['Mean Tumor Volume(mm3)'] = mean_tumor_regimen
mouse_study_stats_df['Median Tumor Volume(mm3)'] = median_tumor_volume
mouse_study_stats_df['Tumor Volume(mm3) Std. Dev.'] = tumor_volume__stddev
mouse_study_stats_df['Tumor Volume(mm3) Std. Err'] = tumor_volume_stderr

#Assemble the resulting series into a single summary DataFrame.

mouse_study_stats_df.head(20)
#mean_tumor_regimen

,Mean Tumor Volume(mm3),Median Tumor Volume(mm3),Tumor Volume(mm3) Std. Dev.,Tumor Volume(mm3) Std. Err
Drug Regimen,,,,
Capomulin,41.963636,42.996848,2.782135,0.879788


# flag-- wrong values

In [18]:
mouse_study_grouped_single = mouse_study_df2.groupby('Drug Regimen').agg({'Tumor Volume (mm3)': ['mean', 'median', 'std','sem']})
mouse_study_grouped_single.head()


Tumor Volume (mm3)                               
                           mean     median       std       sem
Drug Regimen                                                  
Capomulin             41.963636  42.996848  2.782135  0.879788

### Bar and Pie Charts


In [19]:
#Create the first bar chart with the Pandas DataFrame.plot() method

total_timepoints = mouse_study_df2.groupby(['Drug Regimen'])['Timepoint'].counts()
drug_regimen = mouse_study_df2['Drug Regimen']

plt.bar(total_timepoints, drug_regimen)
plt.title('Total number of timepoints for each drug regimen')
plt.xlabel('Drug Regimen')
plt.ylabel('Timepoints')
plt.show()


AttributeError: 'SeriesGroupBy' object has no attribute 'counts'

In [ ]:
#Create the second bar chart with Matplotlib's pyplot methods.

# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.
num_mice_tested = mouse_study_df2.groupby(['Drug Regimen'])['Mouse ID','Timepoint'].value_counts()

num_mice_chart = num_mice_tested.plot(kind='bar', figsize=(6,8))
num_mice_chart.set_xlabel('Drug Regimen')
num_mice_chart.set_ylabel('Number of Mice Tested')
plt.ylim(0,250)
plt.show()
#plt.tight_layout()

# Pie Chart- Flag- off numbers

In [ ]:
#Pie Chart using Matplotlib

# Creating dataset
gender = ['Male', 'Female']
 
my_data = (mouse_study_df2["Sex"].value_counts()/248)*100

# Creating plot
plt.pie(my_data, labels=gender, autopct='%1.1f%%')
plt.title('Percentage Sex of Mice')
plt.axis('equal')
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
# Creating plot Using Pandas

#labels = ["Male", "Female"]
gender_data = (mouse_study_df2["Sex"].value_counts()/248)*100

#print(gender_data_male)

# creating dataframe
gender = gender_data.plot(kind='pie',autopct='%.1f%%',labels=["Male", "Female"])

 
# show plot
#plt.show()

### Quartiles, Outliers, and Boxplots

#Calculate the final tumor volume of each mouse across four of the treatment regimens:  
#Capomulin, Ramicane, Infubinol, and Ceftamin
#Start by getting the last (greatest) timepoint for each mouse
#Merge this group df with the original DataFrame to get the tumor volume at the last timepoint


In [ ]:
# Final Tumor Volume for each Mouse -Capomulin, Ramicane, Infubinol, and Ceftamin

#Create a grouped DataFrame that shows the last (greatest) time point for each mouse. 
#Merge this grouped DataFrame with the original cleaned DataFrame.

final_tumor_volume_df = pd.DataFrame()

In [ ]:
#Create a list that holds the treatment names as well as a second, empty list to hold the tumor volume data.

treatment_names = []
tumor_volume_data = []

In [ ]:
#Loop through each drug in the treatment list, 
#locating the rows in the merged DataFrame that correspond to each treatment. 
#Append the resulting final tumor volumes for each drug to the empty list.

for final_volume in <<list>>:
    tumor_volume_data.append(final_volume)
    


In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.boxplot.html
    
import itertools
tuples = [t for t in itertools.product(range(1000), range(4))]
index = pd.MultiIndex.from_tuples(tuples, names=['lvl0', 'lvl1'])
data = np.random.randn(len(index),4)
df = pd.DataFrame(data, columns=list('ABCD'), index=index)
grouped = df.groupby(level='lvl1')
grouped.boxplot(rot=45, fontsize=12, figsize=(8,10))  

In [ ]:
#Determine outliers by using the upper and lower bounds, and then print the results.
#https://medium.datadriveninvestor.com/finding-outliers-in-dataset-using-python-efc3fce6ce32


outliers=[]
def detect_outlier(data_1):
    #find the mean and standard deviation of the all the data points
    threshold=3
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    
    #find the z score for each of the data point in the dataset
    #if the z score is greater than 3 than we can classify that point as an outlier.
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold:
            outliers.append(y)
    return outliers

Use Matplotlib, generate a box plot that shows the distribution of the final tumor volume for all the mice in each treatment group. Highlight any potential outliers in the plot by changing their color and style.

### Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
 '''df = pd.DataFrame([[5.1, 3.5, 0], [4.9, 3.0, 0], [7.0, 3.2, 1],
                   [6.4, 3.2, 1], [5.9, 3.0, 2]],
                   columns=['length', 'width', 'species'])
>>> ax1 = df.plot.scatter(x='length',
                       y='width',
                       c='DarkBlue')'''